In [1]:
from pathlib import Path

# For a relatively small iid count, create sample inpput files.

iid_count = 1_000

root = Path(r'm:\projects\pedigree')

id_file = root / f'test_sp_{iid_count}.grm.id'
id_file.parent.mkdir(parents=True, exist_ok=True)

index_to_id = [f"fid{i+1} iid{i+1} {i+1}" for i in range(iid_count)]

if not id_file.exists():
    with open(id_file, 'w') as f:
        for i in range(iid_count):
            f.write(f"{index_to_id[i]}\n")

index_to_id[:5]


['fid1 iid1 1', 'fid2 iid2 2', 'fid3 iid3 3', 'fid4 iid4 4', 'fid5 iid5 5']

In [2]:
# make a random iid_count x iid_count matrix with values between 0 and 1. It should be symmetric. Values on the diagonal should be 1.
import numpy as np

# create a random matrix of size iid_count x iid_count
sp = np.random.rand(iid_count, iid_count)/2

# make the matrix symmetric
sp = sp + sp.T - np.diag(sp.diagonal())
np.fill_diagonal(sp, 1)

sp[:5, :5]

array([[1.        , 0.56663475, 0.36093343, 0.70173945, 0.31661688],
       [0.56663475, 1.        , 0.51142044, 0.38745994, 0.38127421],
       [0.36093343, 0.51142044, 1.        , 0.85326691, 0.23566139],
       [0.70173945, 0.38745994, 0.85326691, 1.        , 0.15716772],
       [0.31661688, 0.38127421, 0.23566139, 0.15716772, 1.        ]])

In [ ]:
sp_file = root / f'test_sp_{iid_count}.grm.sp'

if not sp_file.exists():
    with open(sp_file, 'w') as f:
        for i in range(1,iid_count+1):
            for j in range(1,iid_count+1):
